In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sys
import os
import time

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from registration_utilities import *
from metadata import *
from data_manager import *

import matplotlib.pyplot as plt
%matplotlib inline

Setting environment for Gordon


No vtk


In [3]:
stack_fixed = 'MD595'
stack_moving = 'atlasV2'

classifier_setting = 2

# Global Transform - between two annotation volumes

In [56]:
global_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
DataManager.load_global_alignment_parameters(stack_moving=stack_moving, moving_volume_type='annotation',
                                            stack_fixed=stack_fixed, fixed_volume_type='annotation',
                                            global_transform_scheme=global_transform_scheme)

In [84]:
vol_m = DataManager.load_annotation_volume(stack=stack_moving, downscale=32)

volume_m_alignedTo_f = \
transform_volume(vol=vol_m, global_params=global_params, centroid_m=centroid_m, centroid_f=centroid_f,
                  xdim_f=xdim_f, ydim_f=ydim_f, zdim_f=zdim_f)

volume_m_alignedTo_f_fn = DataManager.get_transformed_volume_filepath(stack_m=stack_moving, type_m='annotation',
                                        stack_f=stack_fixed, type_f='annotation',
                                        downscale=32,
                                        global_transform_scheme=global_transform_scheme)

create_if_not_exists(os.path.dirname(volume_m_alignedTo_f_fn))
bp.pack_ndarray_file(volume_m_alignedTo_f, volume_m_alignedTo_f_fn)

In [ ]:
display_volume_sections(volume_m_alignedTo_f)

# Global Transform

In [60]:
warp_setting = 1
trial_idx = 0

In [ ]:
# Load transform parameters

global_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
DataManager.load_alignment_parameters(stack_m=stack_moving, stack_f=stack_fixed,
                                      classifier_setting_m=classifier_setting,
                                      classifier_setting_f=classifier_setting,
                                      warp_setting=warp_setting,
                                      trial_idx=0)

In [31]:
# Transform moving volume, sided, without surround

for structure in all_known_structures_sided:
# for name_s in ['7N_L']:
    
    try:
        t = time.time()

        vol_m = DataManager.load_score_volume(stack=stack_moving, structure=structure, downscale=32)

        volume_m_alignedTo_f = \
        transform_volume(vol=vol_m, global_params=global_params, centroid_m=centroid_m, centroid_f=centroid_f,
                          xdim_f=xdim_f, ydim_f=ydim_f, zdim_f=zdim_f)

        volume_m_alignedTo_f_fn = \
        DataManager.get_transformed_volume_filepath(stack_m=stack_moving, type_m='score',
                                                    stack_f=stack_fixed, type_f='score',
                                                    downscale=32,
                                                    classifier_setting_m=classifier_setting,
                                                    classifier_setting_f=classifier_setting,
                                                    warp_setting=warp_setting,
                                                    structure=structure)

        create_if_not_exists(os.path.dirname(volume_m_alignedTo_f_fn))
        bp.pack_ndarray_file(volume_m_alignedTo_f, volume_m_alignedTo_f_fn)
    
        sys.stderr.write('Transform: %.2f seconds.\n' % (time.time() - t))
    
    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Error transforming volume %s.\n' % structure)

File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_volumes/atlasV2/score_volumes/atlasV2_down32_scoreVolume_sp5.bp
[Errno 2] No such file or directory: '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/atlasV2/score_volumes/atlasV2_down32_scoreVolume_sp5.bp'
Error transforming volume sp5.
File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_volumes/atlasV2/score_volumes/atlasV2_down32_scoreVolume_outerContour.bp
[Errno 2] No such file or directory: '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/atlasV2/score_volumes/atlasV2_down32_scoreVolume_outerContour.bp'
Error transforming volume outerContour.


# Structure-wise Transform

In [53]:
warp_setting = 2
trial_idx = 0

In [2]:
if warp_setting == 1:
    upstream_warp_setting = None
elif warp_setting == 2:
    upstream_warp_setting = 1
else:
    raise Exception('Warp setting not recognized.')
    
if trial_idx in [0, 1]:
    upstream_trial_idx = 0

In [51]:
for structure in all_known_structures_sided:

    # Load local transform parameters
    
    try:

        t = time.time()
        
        local_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
        DataManager.load_alignment_parameters(stack_m=stack_moving, stack_f=stack_fixed,
                                              classifier_setting_m=classifier_setting,
                                              classifier_setting_f=classifier_setting,
                                              warp_setting=warp_setting,
                                              param_suffix=structure,
                                              trial_idx=trial_idx)

        # Read global tx
        global_transformed_moving_structure_vol = \
        DataManager.load_transformed_volume(stack_m=stack_moving, stack_f=stack_fixed,
                                            classifier_setting_m=classifier_setting,
                                            classifier_setting_f=classifier_setting,
                                            warp_setting=upstream_warp_setting, 
                                            trial_idx=upstream_trial_idx,
                                            structure=structure)

        # Transform
        local_transformed_moving_structure_vol = transform_volume(vol=global_transformed_moving_structure_vol, 
                                                 global_params=local_params, 
                                                 centroid_m=centroid_m, centroid_f=centroid_f,
                                                 xdim_f=xdim_f, ydim_f=ydim_f, zdim_f=zdim_f)

        # Save
        local_transformed_moving_structure_fn = \
        DataManager.get_transformed_volume_filepath(stack_m=stack_moving, stack_f=stack_fixed,
                                                    classifier_setting_m=classifier_setting,
                                                    classifier_setting_f=classifier_setting,
                                                    warp_setting=warp_setting,
                                                    trial_idx=trial_idx,
                                                    structure=structure)

        create_if_not_exists(os.path.dirname(local_transformed_moving_structure_fn))
        bp.pack_ndarray_file(local_transformed_moving_structure_vol, local_transformed_moving_structure_fn)
        
        sys.stderr.write('Transform: %.2f seconds.\n' % (time.time() - t))
        
    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Error transforming volume %s.\n' % structure)

Structure-wise transform: 3.49

# Global Transform Reversed

In [4]:
warp_setting = 3
trial_idx = 0

In [6]:
if warp_setting == 1:
    upstream_warp_setting = None
elif warp_setting == 2:
    upstream_warp_setting = 1
elif warp_setting == 3:
    global_warp_setting = 1
    upstream_warp_setting = 2
else:
    raise Exception('Warp setting not recognized.')
    
if trial_idx in [0, 1]:
    upstream_trial_idx = 0

In [7]:
# Load transform parameters

global_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
DataManager.load_alignment_parameters(stack_m=stack_moving, stack_f=stack_fixed,
                                      classifier_setting_m=classifier_setting,
                                      classifier_setting_f=classifier_setting,
                                      warp_setting=global_warp_setting,
                                      trial_idx=trial_idx)

In [ ]:
# for structure in ['PBG_L']:
for structure in all_known_structures_sided:
    
    try:
        
        t = time.time()

        local_transformed_moving_structure_vol = \
            DataManager.load_transformed_volume(stack_m=stack_moving, stack_f=stack_fixed,
                                                classifier_setting_m=classifier_setting,
                                                classifier_setting_f=classifier_setting,
                                                warp_setting=upstream_warp_setting, 
                                                trial_idx=upstream_trial_idx,
                                                structure=structure)

        local_transformed_moving_structure_global_reversed_vol = \
        transform_volume_inverse(vol=local_transformed_moving_structure_vol, 
                                 global_params=global_params, 
                                 centroid_m=centroid_m, centroid_f=centroid_f,
                                 xdim_m=xdim_m, ydim_m=ydim_m, zdim_m=zdim_m)

        local_transformed_moving_structure_global_reversed_fn = \
        DataManager.get_transformed_volume_filepath(stack_m=stack_moving, type_m='score',
                                                        stack_f=stack_fixed, type_f='score',
                                                        downscale=32,
                                                        classifier_setting_m=classifier_setting,
                                                        classifier_setting_f=classifier_setting,
                                                        warp_setting=warp_setting,
                                                        structure=structure)

        create_if_not_exists(os.path.dirname(local_transformed_moving_structure_global_reversed_fn))
        bp.pack_ndarray_file(local_transformed_moving_structure_global_reversed_vol, local_transformed_moving_structure_global_reversed_fn)

        sys.stderr.write('Transform: %.2f seconds.\n' % (time.time() - t))
        
    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Error transforming volume %s.\n' % structure)